# README? NoADME!

In [1]:
! pip install -qU gradio python-dotenv langchain-upstage
! pip install -qU requests
! pip install -qU pymupdf

In [2]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

In [3]:
import warnings

warnings.filterwarnings("ignore")

# Define functions to interact with GitHub

In [4]:
import requests
import json
import os
from datetime import datetime

# GitHub Personal Access Token 및 사용자/저장소 정보 설정
GITHUB_TOKEN = os.environ["GITHUB_TOKEN"]
REPO_OWNER = os.environ["REPO_OWNER"]
REPO_NAME = os.environ["REPO_NAME"]

# 파일 경로 및 내용 설정
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
file_path = f"test/test-result-{timestamp}.md"

# GitHub API URL 설정
GITHUB_API_URL = "https://api.github.com"

# 헤더 설정
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

# 1. 브랜치 생성 함수
def create_branch(branch_name):
    # main 브랜치의 최신 커밋 SHA 가져오기
    response = requests.get(f"{GITHUB_API_URL}/repos/{REPO_OWNER}/{REPO_NAME}/git/ref/heads/main", headers=headers)
    sha = response.json()["object"]["sha"]

    # 새 브랜치 생성
    data = {
        "ref": f"refs/heads/{branch_name}",
        "sha": sha
    }
    response = requests.post(f"{GITHUB_API_URL}/repos/{REPO_OWNER}/{REPO_NAME}/git/refs", headers=headers, data=json.dumps(data))
    if response.status_code == 201:
        print(f"Branch '{branch_name}' created successfully.")
    else:
        print(f"Failed to create branch: {response.json()}")

# 2. 파일 커밋 함수
def commit_file(branch_name, file_path, file_content):
    # 파일을 Base64로 인코딩
    import base64
    encoded_content = base64.b64encode(file_content.encode()).decode()

    # 파일 커밋
    data = {
        "message": f"Add test result file {file_path}",
        "content": encoded_content,
        "branch": branch_name
    }
    response = requests.put(f"{GITHUB_API_URL}/repos/{REPO_OWNER}/{REPO_NAME}/contents/{file_path}", headers=headers, data=json.dumps(data))
    if response.status_code == 201:
        print(f"File '{file_path}' committed successfully.")
    else:
        print(f"Failed to commit file: {response.json()}")

# 3. PR 생성 함수
def create_pull_request(branch_name):
    data = {
        "title": "Add test result file",
        "head": branch_name,
        "base": "main",
        "body": "This PR adds a new test result file."
    }
    response = requests.post(f"{GITHUB_API_URL}/repos/{REPO_OWNER}/{REPO_NAME}/pulls", headers=headers, data=json.dumps(data))
    if response.status_code == 201:
        print("Pull request created successfully.")
    else:
        print(f"Failed to create pull request: {response.json()}")


# Set LLM

In [5]:
import gradio as gr

from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage


llm = ChatUpstage(streaming=True)

# Read a README.md from GitHub

In [6]:
import requests

def fetch_github_readme(readme_raw_url):

    # Fetch the README.md file content
    response = requests.get(readme_raw_url)
    if response.status_code == 200:
        return response.text
    else:
        return f"Error fetching README.md file: {response.status_code}"
    
# Example usage
repo_url = "https://raw.githubusercontent.com/noadme/noadme/main/test/test02.md"  # Replace with your GitHub repo URL
readme = fetch_github_readme(repo_url)
readme

'# NoREADME 시스템 설명\n\n## 개요\nNoREADME는 개발자가 GitHub 저장소에 있는 Markdown(md) 파일을 자동으로 분석하고, Mermaid 다이어그램을 생성하여 해당 파일에 추가하는 시스템입니다. LangChain과 Solar(LLM) 파이프라인을 사용하여 파일을 다운로드하고 분석하며, 엔티티 및 인터랙션을 추출한 후 Mermaid 다이어그램을 생성합니다. 생성된 다이어그램은 GitHub에 커밋되어 사용자가 직접 작성하는 번거로움을 줄여줍니다.\n\n## 기능\n1. **사용자 입력**:\n    - 사용자는 NoREADME 웹 인터페이스를 통해 GitHub md 파일 URL을 입력합니다.\n\n2. **파일 다운로드**:\n    - NoREADME 서버는 입력된 URL을 받아 LangChain 파이프라인을 시작합니다.\n    - LangChain이 URL을 통해 md 파일을 GitHub에서 다운로드합니다.\n\n3. **컨텍스트 검색 및 텍스트 분석**:\n    - 다운로드된 파일 콘텐츠는 LangChain 및 Solar(LLM)을 통해 분석됩니다.\n    - 관련 컨텍스트를 검색하고 파일 콘텐츠를 분석하여 엔티티와 인터랙션을 추출합니다.\n\n4. **Mermaid 다이어그램 생성**:\n    - LangChain과 Solar(LLM)이 추출된 엔티티와 인터랙션을 기반으로 다이어그램을 생성합니다.\n    - 생성된 다이어그램은 검증됩니다.\n\n5. **GitHub에 커밋**:\n    - 최종적으로 NoREADME 서버는 생성된 다이어그램을 원본 md 파일에 추가하고, GitHub에 커밋합니다.\n    - 업데이트된 파일은 사용자가 확인할 수 있습니다.\n\n이 시스템은 개발자가 반복적으로 README.md 파일을 작성하는 번거로움을 줄여주어 효율적인 작업 환경을 제공합니다.\n'

# Set knowledge base of Mermaid syntax

TBD

# 2-shot prompt to get mermaid syntax

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import UpstageGroundednessCheck


def gc(aimsg, hmsg):
    groundedness_check = UpstageGroundednessCheck()

    answer = aimsg
    gc_result = groundedness_check.invoke({"context": hmsg, "answer": answer})

    print(gc_result)
    if gc_result.lower().startswith("grounded"):
        return "[ Groundedness check passed ] "
    else:
        return "[ Groundedness check failed ]"


prompt_template = PromptTemplate.from_template(
    """
    Q: Please extract user and participants's name  from the following text. I am a user. Then, organize the action relationships between me and the participants. 

    ---
    We are a team of 5 members called Krispy Team.
    Dr. Kim Yoon-gon gave me coffee. I said thank you.
    Student Jae-min also received coffee from Dr. Kim Yoon-gon. Dr. Son Seok-ho is explaining the team project topic now.
    Student Sung-hoon is preparing presentation materials for other team members.

    A:
    ```mermaid
    sequenceDiagram
        actor User
        participant Kim Yoon-gon
        participant Jae-min
        participant Son Seok-ho
        participant Sung-hoon
        participant Other team members

        User ->> Kim Yoon-gon : Received coffee and said thank you
        Kim Yoon-gon ->> Jae-min : Gave coffee
        Son Seok-ho ->> Team : Explained the project topic
        Sung-hoon ->> Other team members : Prepared presentation materials
    ```

    Q: Please extract the user's name and participants' names from the following text. The user is the person who is writing this prompt. Then, organize the action relationships between the user and the participants into a sequence diagram using mermaid syntax.

    ---
    {text}
    """
)

llm = ChatUpstage()
chain = prompt_template | llm | StrOutputParser()
# content = "We are a team of four.I am currently creating a template to give to Dr. Kim Yoon-gon.Also, Student Sung-hoon is creating an RAG to give to Dr. Kim Yoon-gon. The other two members are discussing ideas with each other."
answer = chain.invoke({"text": readme})

ret = gc(answer, readme)    
ans = answer + "\n" + ret
print(ans)



grounded
A:
```mermaid
sequenceDiagram
    participant User
    participant NoREADME Web Interface
    participant NoREADME Server
    participant LangChain
    participant Solar
    participant GitHub

    User ->> NoREADME Web Interface: 입력 URL 제공
    NoREADME Web Interface ->> NoREADME Server: URL 전송
    NoREADME Server ->> LangChain: 파이프라인 시작
    NoREADME Server ->> GitHub: md 파일 다운로드
    LangChain ->> Solar: 파일 분석
    Solar ->> LangChain: 엔티티 및 인터랙션 추출
    LangChain ->> NoREADME Server: 다이어그램 생성
    NoREADME Server ->> GitHub: 다이어그램 커밋
    GitHub ->> User: 업데이트된 파일 확인
```
[ Groundedness check passed ] 


# Open a PR

In [127]:
# Assign the result (answer) to the file_content
file_content = answer
branch_name = f"test-result-{timestamp}"
create_branch(branch_name)
commit_file(branch_name, file_path, file_content)
create_pull_request(branch_name)

Branch 'test-result-20240706155053' created successfully.
File 'test/test-result-20240706155053.md' committed successfully.
Pull request created successfully.
